# Digital House - Trabajo Práctico Nº1
## Desafío 1. Análisis exploratorio de un dataset de precios de propiedades
Grupo Nª2
Alumnos:
* Alberto Paparelli
* André Song
* Marianela Riz
* Gloria Caravajal
* Franco Emanuel Rodriguez
* Iván Axel Schweizer

Objetivos:
* Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.
* Realizar un análisis descriptivo de las principales variables.
* Crear nuevas columnas a partir de las características dadas que puedan tener valor
predictivo.

In [9]:
# Lo primero que hacemos es importar todas las librerias necesarias.
import pandas as pd

In [10]:
# Abrimos el dataset y hacemos un chequeo rápido para saber cantidad de registros y miramos la composición del mismo.
filename = "../dataset/properati.csv"
data = pd.read_csv(filename)

display(data.sample(n=6))
print(f"(Cantidad de Filas, Cantidad de columnas) -> {data.shape}")

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
102596,102596,sell,house,Villa Luzuriaga,|Argentina|Bs.As. G.B.A. Zona Oeste|La Matanza...,Argentina,Bs.As. G.B.A. Zona Oeste,3427406.0,NaN,NaN,...,67.0,238.636364,1567.164179,NaN,NaN,NaN,http://www.properati.com.ar/1bqsq_venta_casa_v...,Casa sobre lote propio de 10 x 44 ubicada en V...,Casa - Villa Luzuriaga,https://thumbs4.properati.com/7/7zymjeLx5TnZIH...
80689,80689,sell,house,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,NaN,NaN,...,150.0,1196.808511,1500.000000,NaN,NaN,NaN,http://www.properati.com.ar/1ax57_venta_casa_c...,NUEVAS RAÍCES presenta CASAS CALYPSO.5 opcione...,Casa de diseño de ALTA GAMA en Miradores de M...,https://thumbs4.properati.com/2/hIVYoOcLHKmZvF...
55434,55434,sell,apartment,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.5711496,-58.4232966",-34.571150,...,70.0,2642.857143,2642.857143,NaN,3.0,NaN,http://www.properati.com.ar/19p3h_venta_depart...,Semipiso en una zona excelente de Alto Palermo...,Semipiso de 3 ambientes en excelente zona de A...,https://thumbs4.properati.com/8/U6GEloIbLrHyiV...
42115,42115,sell,house,José C Paz,|Argentina|Bs.As. G.B.A. Zona Norte|José C Paz|,Argentina,Bs.As. G.B.A. Zona Norte,NaN,"-34.4955928,-58.7653691",-34.495593,...,70.0,264.705882,1414.285714,NaN,NaN,NaN,http://www.properati.com.ar/18vsu_venta_casa_j...,"Venta de Casa 3 AMBIENTES, José C PazChalet co...",CASA EN VENTA,https://thumbs4.properati.com/9/fMPx9Jr6JwwbEU...
7544,7544,sell,apartment,Monserrat,|Argentina|Capital Federal|Monserrat|,Argentina,Capital Federal,3430570.0,"-34.6177306,-58.3830253",-34.617731,...,34.0,NaN,2235.294118,NaN,1.0,NaN,http://www.properati.com.ar/15y6q_venta_depart...,Monoambiente - Al frenteCaracterísticasEl edif...,Monoambiente a estrenar/ San Telmo,https://thumbs4.properati.com/2/u0hoD7jMlw--fN...
84056,84056,sell,apartment,Córdoba,|Argentina|Córdoba|,Argentina,Córdoba,3860255.0,NaN,NaN,...,47.0,1112.003793,24468.085106,NaN,NaN,NaN,http://www.properati.com.ar/1b19x_venta_depart...,"Departamento ubicado en 2º Piso,tipologia desi...",Oportunidad UNICA en FIRMUS. VENTA DPTO NUEVA ...,https://thumbs4.properati.com/6/AW2LWEmSPDWqcc...


(Cantidad de Filas, Cantidad de columnas) -> (121220, 26)


# TODO 
completar análisis de columnas: que columnas hay y que significan cada una.


Analizamos contenido de las columnas que creemos que pueden eliminarse.

In [24]:
# Columna operation: revisamos que opciones tiene, o si se repite lo mismo.
print(f'Cantidad de registros con NaN: {data.operation.isnull().sum()}')
print("Contamos cantidad de registros unicos:")
print(data.groupby(['operation']).size().reset_index(name='count'))


Cantidad de registros con NaN: 0
  operation   count
0      sell  121220


Detalle de análisis de columnas:
* `operation` -> Vemos que no tiene NaNs y solo tiene la opción `sell`. Por este motivo la vamos a eliminar.